In [40]:
import pickle
import os
import sys
sys.path.append("../")
from od.social.group import SocialGroup
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,NET_TIMEOUT_SECONDS
from od.env.station import BS_PRESET
import od.misc.statistic as oms

In [41]:
print(os.getcwd())
statistics = []
for i,alloc_type in enumerate([ResourceAllocatorType.NOMA_OPT,ResourceAllocatorType.NOMA_APR]):
    interest_config = InterestConfig(True,alloc_type,True,1.3,7)
    with open("../data/TVAL/{}/object.pickle".format(interest_config.folder()), "rb") as file:
        statistics.append(pickle.load(file))
        statistics[-1].nft_traffic = {}

/home/OD/repos/ProjectSumo/ipynb


In [42]:
statistics

In [43]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV
def RecordPrinter(header):
    print("time at:{}".format(header.at))
    print("bs ot:{}".format(header.is_bs_ot))
    print("src ot:{}".format(header.is_src_ot))
    print("time bs drop:{}".format(header.time_bs_drop))
    print("time bs serv:{}".format(header.time_bs_serv))
    print("time bs tx:{}".format(header.time_bs_tx))
    print("time bs txq:{}".format(header.time_bs_txq))
    print("time bs trip:{}".format(header.time_veh_trip))

In [44]:
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2, UMI-1, UMI-2, UMI-3, UMI-4, UMI-5]

In [45]:
for statistic in statistics:
    for sg in statistic.social_group:
        for record in statistic.sg_header[sg].values():
            for bs in GV.NET_STATION_CONTROLLER:
                if(len(record.time_bs_txq[bs])==0):
                    continue
                if(record.time_bs_txq[bs][-1][1] - record.at >= NET_TIMEOUT_SECONDS):
                    record.is_bs_ot[bs] = True

In [46]:
for i,statistic in enumerate(statistics):
    print("====={}======".format(i))
    for sg in statistic.social_group:
        ot_count = 0 
        for header, record in statistic.sg_header[sg].items():
            if record.is_src_ot > 0:
                ot_count +=1
            elif (True in [record.is_bs_ot[i] for i in GV.NET_STATION_CONTROLLER]):
                ot_count +=1

        print("{}: {} => {}".format(sg,ot_count, ot_count / max(len(statistic.sg_header[sg]),1)))
            

=====0======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 0 => 0.0
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 122 => 0.6455026455026455
=====1======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 999 => 0.6294896030245747
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 189 => 1.0


In [47]:
class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len =  195-185 if self.time_serv == -1 else round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [48]:
target_header = "flow_4.3-198"
# target_header = "flow_0.58-99"
for statistic in statistics:
    header_in_sg_list = [sg.fname for sg in statistic.social_group   if(target_header in statistic.sg_header[sg])]
    print("In {}".format(header_in_sg_list))
    print("C?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)))
    print("G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.GENERAL)))
    print("C2G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.C2G)))

In ['DYN_SG_3']
C?:False
G?:True
C2G?:False
In ['DYN_SG_3']
C?:False
G?:True
C2G?:False


In [49]:
for statistic in statistics:
    for sg in statistic.social_group:
        if target_header in statistic.sg_header[sg]:
            header = statistic.sg_header[sg][target_header]
    print("======={}========".format(statistic))
    RecordPrinter(header)

=======<od.misc.statistic.StatisticRecorder object at 0x7f0140fdb7c0>========
time at:187.105
bs ot:[False, False, False, False, False, False, False]
src ot:False
time bs drop:[-1, -1, -1, -1, -1, -1, -1]
time bs serv:[187.106, 189.225, -1, -1, -1, -1, -1]
time bs tx:[[[187.106, 187.107]], [[189.225, 189.226]], [], [], [], [], []]
time bs txq:[[[187.106, 187.106]], [[187.106, 189.225]], [], [], [], [], []]
time bs trip:{'flow_0.32': 0.0020000000000095497, 'flow_0.35': 0.0020000000000095497, 'flow_0.39': 0.0020000000000095497, 'flow_0.42': 0.0020000000000095497, 'flow_0.53': 0.0020000000000095497, 'flow_0.54': 0.0020000000000095497, 'flow_0.56': 0.0020000000000095497, 'flow_0.61': 0.0020000000000095497, 'flow_1.7': 0.0020000000000095497, 'flow_9.7': 0.0020000000000095497, 'flow_2.56': 2.1210000000000093, 'flow_2.57': 2.1210000000000093, 'flow_2.80': 2.1210000000000093}
=======<od.misc.statistic.StatisticRecorder object at 0x7f01407b8e50>========
time at:187.105
bs ot:[False, True, False

In [52]:
bs_index = 1
social_groups = statistics[0].social_group
print(social_groups)
# app_stats_fn = lambda x: statistic.sg_header[social_groups[3]]
app_stats_fn = lambda x: statistic.ExtractNetworkTraffic(NetFlowType.C2G)
# app_stats_fn = lambda x: x.sg_header[SocialGroup.RCWS]

[<CRASH(QoS:0, Gid:0)>, <RCWS(QoS:1, Gid:1)>, <DYN_SG_0(QoS:1, Gid:2)>, <DYN_SG_1(QoS:1, Gid:3)>, <DYN_SG_2(QoS:1, Gid:4)>, <DYN_SG_3(QoS:1, Gid:5)>, <DYN_SG_4(QoS:1, Gid:6)>, <DYN_SG_5(QoS:1, Gid:7)>, <DYN_SG_6(QoS:1, Gid:8)>, <DYN_SG_7(QoS:1, Gid:9)>, <DYN_SG_8(QoS:1, Gid:10)>]


In [53]:
bs_serv_list = [[] for _ in statistics]
bs_time_gap = [{} for _ in statistics]
for i, statistic in enumerate(statistics):
    for header, record in app_stats_fn(statistic).items():
        if(len(record.time_bs_txq[bs_index]) > 0):
            bs_serv_list[i].append(Helper(header, record, bs_index))
    bs_serv_list[i].sort(key=lambda x: (x.time_serv * 1000) * 10000000 + x.bits)
    for helper in bs_serv_list[i]:
        at = str(helper.time_gen)
        to = str(helper.time_serv)
        if(to not in bs_time_gap[i]):
             bs_time_gap[i][to] = {"need": 0, "give": 0}
        if(at not in bs_time_gap[i]):
             bs_time_gap[i][at] = {"need": 0, "give": 0}
        bs_time_gap[i][to]["give"]+=helper.bits
        bs_time_gap[i][at]["need"]+=helper.bits
    
   


In [54]:
for time in sorted(bs_time_gap[1].keys()):
    print(time, bs_time_gap[1][time])

186.0 {'need': 29544, 'give': 0}
186.001 {'need': 54184, 'give': 21872}
186.002 {'need': 88232, 'give': 22712}
186.003 {'need': 100800, 'give': 22040}
186.004 {'need': 92168, 'give': 22232}
186.005 {'need': 60032, 'give': 24168}
186.006 {'need': 51496, 'give': 23552}
186.007 {'need': 6816, 'give': 13368}
186.008 {'need': 0, 'give': 6952}
186.009 {'need': 0, 'give': 20392}
186.01 {'need': 0, 'give': 24240}
186.011 {'need': 0, 'give': 28248}
186.012 {'need': 0, 'give': 48928}
186.013 {'need': 0, 'give': 47384}
186.014 {'need': 5088, 'give': 22088}
186.015 {'need': 0, 'give': 8032}
186.016 {'need': 13704, 'give': 13992}
186.017 {'need': 44440, 'give': 21672}
186.018 {'need': 62208, 'give': 16272}
186.019 {'need': 75808, 'give': 16816}
186.02 {'need': 119856, 'give': 11896}
186.021 {'need': 56280, 'give': 23912}
186.022 {'need': 55144, 'give': 15688}
186.023 {'need': 48992, 'give': 20696}
186.024 {'need': 0, 'give': 41736}
186.025 {'need': 0, 'give': 30472}
186.026 {'need': 0, 'give': 2436

In [55]:
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    data = []
    for i in range(2):
        if(time in bs_time_gap[i]):
            data.append(bs_time_gap[i][time]["need"])
            data.append(bs_time_gap[i][time]["give"])
        else:
            data.append(0)
            data.append(0)
    if(not sum(data)==0):
        print(time, data)


186.0 [29544, 0, 29544, 0]
186.001 [54184, 29544, 54184, 21872]
186.002 [88232, 37768, 88232, 22712]
186.003 [100800, 51144, 100800, 22040]
186.004 [92168, 38440, 92168, 22232]
186.005 [60032, 44632, 60032, 24168]
186.006 [51496, 47296, 51496, 23552]
186.007 [6816, 42792, 6816, 13368]
186.008 [0, 41608, 0, 6952]
186.009 [0, 49024, 0, 20392]
186.01 [0, 46120, 0, 24240]
186.011 [0, 48088, 0, 28248]
186.012 [0, 6816, 0, 48928]
186.013 [0, 0, 0, 47384]
186.014 [5088, 0, 5088, 22088]
186.015 [0, 5088, 0, 8032]
186.016 [13704, 0, 13704, 13992]
186.017 [44440, 13704, 44440, 21672]
186.018 [62208, 37832, 62208, 16272]
186.019 [75808, 40848, 75808, 16816]
186.02 [119856, 46544, 119856, 11896]
186.021 [56280, 40696, 56280, 23912]
186.022 [55144, 49464, 55144, 15688]
186.023 [48992, 38368, 48992, 20696]
186.024 [0, 51088, 0, 41736]
186.025 [0, 37968, 0, 30472]
186.026 [0, 48072, 0, 24368]
186.027 [0, 49400, 0, 21960]
186.028 [0, 22448, 0, 44768]
186.029 [0, 0, 0, 44408]
186.03 [5528, 0, 5528, 387

In [56]:

remain = [0 for _ in statistics]
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    for i in range(2):
        if(time in bs_time_gap[i]):
            helper = bs_time_gap[i][time]
            remain[i] += (helper["need"] - helper["give"])
    if(not sum(remain)==0):
        print(time, remain, remain[0]-remain[1])


186.0 [29544, 29544] 0
186.001 [54184, 61856] -7672
186.002 [104648, 127376] -22728
186.003 [154304, 206136] -51832
186.004 [208032, 276072] -68040
186.005 [223432, 311936] -88504
186.006 [227632, 339880] -112248
186.007 [191656, 333328] -141672
186.008 [150048, 326376] -176328
186.009 [101024, 305984] -204960
186.01 [54904, 281744] -226840
186.011 [6816, 253496] -246680
186.012 [0, 204568] -204568
186.013 [0, 157184] -157184
186.014 [5088, 140184] -135096
186.015 [0, 132152] -132152
186.016 [13704, 131864] -118160
186.017 [44440, 154632] -110192
186.018 [68816, 200568] -131752
186.019 [103776, 259560] -155784
186.02 [177088, 367520] -190432
186.021 [192672, 399888] -207216
186.022 [198352, 439344] -240992
186.023 [208976, 467640] -258664
186.024 [157888, 425904] -268016
186.025 [119920, 395432] -275512
186.026 [71848, 371064] -299216
186.027 [22448, 349104] -326656
186.028 [0, 304336] -304336
186.029 [0, 259928] -259928
186.03 [5528, 226720] -221192
186.031 [0, 191472] -191472
186.032

In [ ]:
for i in range(2):
    trip_times = list(map(lambda x: x.time_len, bs_serv_list[i]))
    print(sum(trip_times) / len(trip_times))


In [ ]:
for record in bs_serv_list[1]:
    print(record)